#RAG_student_handbook
create ->chromadb \
run roberta model

PIP's

In [ ]:
%pip install sentence-transformers transformers==4.32.0 tf-keras langchain_community langchain_text_splitters chromadb torch

In [ ]:
%pip install --upgrade transformers sentence-transformers
%pip install pypdf numpy

  Using cached transformers-4.51.0-py3-none-any.whl.metadata (38 kB)
  Using cached sentence_transformers-4.0.2-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.51.0-py3-none-any.whl (10.4 MB)
Using cached sentence_transformers-4.0.2-py3-none-any.whl (340 kB)
Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.32.0
    Uninstalling transformers-4.32.0:
      Successfully uninstalled transformers-4.32.0
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.6.1
    Uninstalling sentence-transformers-2.6.1:
      Successfully 

In [ ]:
%pip install chromadb


In [ ]:
!pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.4.0
accelerate                               1.5.2
aiohappyeyeballs                         2.6.1
aiohttp                                  3.11.15
aiosignal                                1.3.2
alabaster                                1.0.0
albucore                                 0.0.23
albumentations                           2.0.5
ale-py                                   0.10.2
altair                                   5.5.0
annotated-types                          0.7.0
anyio                                    4.9.0
argon2-cffi                              23.1.0
argon2-cffi-bindings                     21.2.0
array_record                             0.7.1
arviz                                    0.21.0
asgiref                                  3.8.1
astropy                                  7.0.1
astropy-iers-data                    

Import the necessary libraries

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import chromadb
from chromadb.config import Settings

Load and cut a PDF document into pieces of text

In [ ]:
loader = PyPDFLoader("BSc_Student Handbook_EN_M.pdf")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=150)
chunks = text_splitter.split_documents(docs)
text_lines = [chunk.page_content for chunk in chunks]

In [ ]:
# Fonction pour générer les embeddings
def encoding_text(text_lines):
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

    if not text_lines:
        raise ValueError("La liste de textes est vide.")

    embeddings = embedding_model.encode(text_lines)
    return embeddings

Create chroma_db

In [ ]:
import chromadb
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="rag_collection")


In [ ]:
# Ajouter les documents dans ChromaDB
for i, line in enumerate(text_lines):
    collection.add(
        ids=[str(i)],
        embeddings=[encoding_text([line])[0].tolist()],  # Convertir en liste
        metadatas=[{"text": line}]
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Run the model with a question

In [ ]:
question = "Who is Simon?"

In [ ]:
# Search for relevant passages in ChromaDB
search_res = collection.query(
    query_embeddings=[encoding_text([question])[0].tolist()],
    n_results=3
)
results = search_res["metadatas"]
context = "\n".join([hit["text"] for sublist in results for hit in sublist if "text" in hit])

In [ ]:
# Creating the prompt

PROMPT = """
Use the information enclosed in <context> tags to provide an answer to the
question enclosed in <question> tags.
<context>
{context}
</context>
<question>
{question}
</question>
Réponse:
"""
prompt = PROMPT.format(context=context, question=question)

In [ ]:
from transformers import pipeline

# question-answering model
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# answer
response = qa_pipeline(question=question, context=context)
print("Generated answer :", response["answer"])


OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.